In [2]:
import re
import pandas as pd

In [14]:

def preprocessing_dataset(dataset):
  """ 처음 불러온 csv 파일을 원하는 형태의 DataFrame으로 변경 시켜줍니다."""
  subject_entity = []
  object_entity = []
  subject_type= []
  object_type= []
  subject_start= []
  object_start= []
  subject_end= []
  object_end= []
  

  for i,j in zip(dataset['subject_entity'], dataset['object_entity']):
    # i = i[1:-1].split(',')[0].split(':')[1]
    sub_entity = eval(i)
    ob_entity = eval(j)
    sub_word= sub_entity['word']
    sub_type = sub_entity['type']
    # j = j[1:-1].split(',')[0].split(':')[1]
    ob_word = ob_entity['word']
    ob_type = ob_entity['type']

    ob_start = ob_entity['start_idx']
    sub_start = sub_entity['start_idx']

    ob_end = ob_entity['end_idx']
    sub_end = sub_entity['end_idx']


    subject_entity.append(sub_word)
    object_entity.append(ob_word)
    subject_type.append(sub_type)
    object_type.append(ob_type)
    subject_start.append(sub_start)
    object_start.append(ob_start)
    subject_end.append(sub_end)
    object_end.append(ob_end)
  
  out_dataset = pd.DataFrame({'id':dataset['id'], 'sentence':dataset['sentence'],'subject_entity':subject_entity,'object_entity':object_entity,'label':dataset['label'],
                              'subject_type' : subject_type, 'object_type': object_type,'sub_start' : subject_start,'ob_start':object_start,'sub_end' :subject_end ,'ob_end': object_end})
  out_dataset.id = out_dataset.index
  return out_dataset


  
def load_data(dataset_dir):
  """ csv 파일을 경로에 맡게 불러 옵니다. """
  pd_dataset = pd.read_csv(dataset_dir)
  dataset = preprocessing_dataset(pd_dataset)
  
  return dataset

### 1. 기본 베이스 모델

In [12]:
def tokenized_dataset(dataset):
  """ tokenizer에 따라 sentence를 tokenizing 합니다."""
  concat_entity = []
  for e01, e02 in zip(dataset['subject_entity'], dataset['object_entity']):
    temp = ''
    temp = e01 + ' [SEP] ' + e02
    concat_entity.append(temp)
  
  print(concat_entity[:5])
  print(dataset['sentence'].head())
  # dataset['sentence'] = dataset['id'].apply(lambda x: change_word(dataset.sentence.loc[x],dataset.subject_entity.loc[x],
  #  
train_dataset = load_data("../data/dataset/train/train_totalX3.csv")
tokenized_train = tokenized_dataset(train_dataset)

['에스타니슬라오 피게라스 [SEP] 스페인', '고등 연구 계획국 [SEP] 미국 국방부', '울버햄튼 [SEP] 잉글랜드', 'K3리그 베이직 [SEP] K3리그 어드밴스', '로날트 더 부르 [SEP] 네덜란드']
0    1873년 스페인의 왕정이 폐지되고 공화국이 설립되자, 아마데오 1세는 퇴위당하고 ...
1    인터넷은 1969 년 미국 국방부 부하의 고등 연구 계획국 (ARPA)에 의해 제안...
2    손흥민의 토트넘은 15일 오후 11시 영국 울버햄튼의 몰리뉴 스타디움에서 울버햄튼 ...
3    더불어 2017 시즌을 기점으로 K3리그가 K3리그 어드밴스로 개편되고, 하위 리그...
4    또한 프랑크 더 부르의 쌍둥이 동생인 로날트 더 부르도 네덜란드 축구 국가대표팀과 ...
Name: sentence, dtype: object


### 2. 단어를 토큰으로 바꾸기


In [13]:

def change_word(sentence, sub_word, sub_type , ob_word, ob_type):
  sentence = re.sub(rf'{sub_word}',f'<{sub_type}>',sentence)
  sentence = re.sub(rf'{ob_word}',f'<{ob_type}>',sentence)

  return sentence

def tokenized_dataset(dataset):
  """ tokenizer에 따라 sentence를 tokenizing 합니다."""
  concat_entity = []
  for e01, e02 in zip(dataset['subject_entity'], dataset['object_entity']):
    temp = ''
    temp = e01 + ' [SEP] ' + e02
    concat_entity.append(temp)
  
  dataset['sentence'] = dataset['id'].apply(lambda x: change_word(dataset.sentence.loc[x],dataset.subject_entity.loc[x],
                                                                  dataset.subject_type.loc[x],dataset.object_entity.loc[x], dataset.object_type.loc[x]))
  
  print(concat_entity[:5])
  print(dataset['sentence'].head())
                                  
train_dataset = load_data("../data/dataset/train/train_totalX3.csv")
tokenized_train = tokenized_dataset(train_dataset)                                

['에스타니슬라오 피게라스[SEP]스페인', '고등 연구 계획국[SEP]미국 국방부', '울버햄튼[SEP]잉글랜드', 'K3리그 베이직[SEP]K3리그 어드밴스', '로날트 더 부르[SEP]네덜란드']
0    1873년 <LOC>의 왕정이 폐지되고 공화국이 설립되자, 아마데오 1세는 퇴위당하...
1    인터넷은 1969 년 <ORG> 부하의 <ORG> (ARPA)에 의해 제안된 BBN...
2    손흥민의 토트넘은 15일 오후 11시 영국 <ORG>의 몰리뉴 스타디움에서 <ORG...
3    더불어 2017 시즌을 기점으로 K3리그가 <ORG>로 개편되고, 하위 리그로서 <...
4    또한 프랑크 더 부르의 쌍둥이 동생인 <PER>도 <LOC> 축구 국가대표팀과 유럽...
Name: sentence, dtype: object


### 3. 인덱스 스타트 번호 넣기

In [15]:

def change_word(sentence, sub_word, sub_type , ob_word, ob_type):
  sentence = re.sub(rf'{sub_word}',f'<{sub_type}>',sentence)
  sentence = re.sub(rf'{ob_word}',f'<{ob_type}>',sentence)

  return sentence

def tokenized_dataset(dataset):
  """ tokenizer에 따라 sentence를 tokenizing 합니다."""
  concat_entity = []
  for e01, e02, s01, s02 in zip(dataset['subject_entity'], dataset['object_entity'], dataset['sub_start'], dataset['ob_start']):
    temp = ''
    temp = e01 + ' [SEP] '+ f'{s01}' + ' [SEP] ' + e02+ ' [SEP] '+ f'{s02}'
    concat_entity.append(temp)
  
  # dataset['sentence'] = dataset['id'].apply(lambda x: change_word(dataset.sentence.loc[x],dataset.subject_entity.loc[x],
  #                                                                 dataset.subject_type.loc[x],dataset.object_entity.loc[x], dataset.object_type.loc[x]))
  
  print(concat_entity[:5])
  print(dataset['sentence'].head())
                                  
train_dataset = load_data("../data/dataset/train/train_totalX3.csv")
tokenized_train = tokenized_dataset(train_dataset)                                

['에스타니슬라오 피게라스 [SEP] 46 [SEP] 스페인 [SEP] 6', '고등 연구 계획국 [SEP] 23 [SEP] 미국 국방부 [SEP] 12', '울버햄튼 [SEP] 24 [SEP] 잉글랜드 [SEP] 60', 'K3리그 베이직 [SEP] 49 [SEP] K3리그 어드밴스 [SEP] 24', '로날트 더 부르 [SEP] 21 [SEP] 네덜란드 [SEP] 31']
0    1873년 스페인의 왕정이 폐지되고 공화국이 설립되자, 아마데오 1세는 퇴위당하고 ...
1    인터넷은 1969 년 미국 국방부 부하의 고등 연구 계획국 (ARPA)에 의해 제안...
2    손흥민의 토트넘은 15일 오후 11시 영국 울버햄튼의 몰리뉴 스타디움에서 울버햄튼 ...
3    더불어 2017 시즌을 기점으로 K3리그가 K3리그 어드밴스로 개편되고, 하위 리그...
4    또한 프랑크 더 부르의 쌍둥이 동생인 로날트 더 부르도 네덜란드 축구 국가대표팀과 ...
Name: sentence, dtype: object


### 3. 스타트 번호, 끝번호 둘다 넣기

In [16]:

def change_word(sentence, sub_word, sub_type , ob_word, ob_type):
  sentence = re.sub(rf'{sub_word}',f'<{sub_type}>',sentence)
  sentence = re.sub(rf'{ob_word}',f'<{ob_type}>',sentence)

  return sentence

def tokenized_dataset(dataset):
  """ tokenizer에 따라 sentence를 tokenizing 합니다."""
  concat_entity = []
  for e01, e02, s01, s02, d01, d02 in zip(dataset['subject_entity'], dataset['object_entity'], dataset['sub_start'], dataset['ob_start'], dataset['sub_end'], dataset['ob_end']):
    temp = ''
    temp = e01 + ' [SEP] '+ f'{s01} {d01}' + ' [SEP] ' + e02+ ' [SEP] '+ f'{s02} {d02}'
    concat_entity.append(temp)
  
  # dataset['sentence'] = dataset['id'].apply(lambda x: change_word(dataset.sentence.loc[x],dataset.subject_entity.loc[x],
  #                                                                 dataset.subject_type.loc[x],dataset.object_entity.loc[x], dataset.object_type.loc[x]))
  
  print(concat_entity[:5])
  print(dataset['sentence'].head())
                                  
train_dataset = load_data("../data/dataset/train/train_totalX3.csv")
tokenized_train = tokenized_dataset(train_dataset)                                

['에스타니슬라오 피게라스 [SEP] 46 57 [SEP] 스페인 [SEP] 6 8', '고등 연구 계획국 [SEP] 23 31 [SEP] 미국 국방부 [SEP] 12 17', '울버햄튼 [SEP] 24 27 [SEP] 잉글랜드 [SEP] 60 63', 'K3리그 베이직 [SEP] 49 56 [SEP] K3리그 어드밴스 [SEP] 24 32', '로날트 더 부르 [SEP] 21 28 [SEP] 네덜란드 [SEP] 31 34']
0    1873년 스페인의 왕정이 폐지되고 공화국이 설립되자, 아마데오 1세는 퇴위당하고 ...
1    인터넷은 1969 년 미국 국방부 부하의 고등 연구 계획국 (ARPA)에 의해 제안...
2    손흥민의 토트넘은 15일 오후 11시 영국 울버햄튼의 몰리뉴 스타디움에서 울버햄튼 ...
3    더불어 2017 시즌을 기점으로 K3리그가 K3리그 어드밴스로 개편되고, 하위 리그...
4    또한 프랑크 더 부르의 쌍둥이 동생인 로날트 더 부르도 네덜란드 축구 국가대표팀과 ...
Name: sentence, dtype: object
